In [1]:
import sys
import os
import numpy as np
from process_bigraph import Composite
from biosimulator_processes import CORE
sys.path.insert(0, '..')


model_fp = './examples/smoldyn/MinE.txt'
os.path.exists(model_fp)
composition = Composite(
    config={
        'state': {
            'smoldyn': {
                '_type': 'process',
                'address': 'local:smoldyn-process',
                'config': {'model_filepath': model_fp},
                'inputs': {
                    'species_counts': ['species_counts'],
                    'molecules': ['molecules']
                },
                'outputs': {
                    'species_counts': ['species_counts'],
                    'molecules': ['molecules']
                }
            },
            # 'emitter': {
            #     '_type': 'step',
            #     'address': 'local:ram-emitter',
            #     'config': {
            #         'emit': {
            #             'species_counts': 'tree[int]',
            #             'molecules': 'tree[float]'
            #         }
            #     },
            #     'inputs': {
            #         'species_counts': ['species_counts'],
            #         'molecules': ['molecules']
            #     }
            # }
        }
    }, 
    core=CORE
)

Available processes:
[ 'console-emitter',
  'ram-emitter',
  'composite',
  'cobra-process',
  'copasi-process',
  'tellurium-process',
  'utc-amici',
  'utc-copasi',
  'utc-tellurium',
  'smoldyn-process',
  'copasi-step',
  'tellurium-step',
  'amici-step',
  'plotter',
  'plotter2d',
  'utc-comparator',
  'simple-neuron',
  'simple-neuron-network']
INITAL: {'MinDMinE': 0, 'MinD_ADP': 0, 'MinD_ATP': 4000, 'MinE': 1400}


In [2]:
results = {}
smoldyn_process = composition.state['smoldyn']['instance']
for n in range(3):
    if n == 0:
        init_state = smoldyn_process.initial_state()
        n += 1
    else:
        init_state = {
            'species_counts': composition.state['species_counts'],
            'molecules': composition.state['molecules']
        }
    r = smoldyn_process.update(init_state, n)
    results[n] = r

INITAL: {'MinDMinE': 0, 'MinD_ADP': 0, 'MinD_ATP': 4000, 'MinE': 1400}


Libsmoldyn notification from smolRunSim: Simulation complete
Libsmoldyn notification from smolRunSim: Simulation complete
Libsmoldyn notification from smolRunSim: Simulation complete


In [3]:
results

{1: {'species_counts': {'MinDMinE': 1805,
   'MinD_ADP': 828,
   'MinD_ATP': -3967,
   'MinE': -33},
  'molecules': {'8afd637e-16ce-49d3-a6d9-213b3217a268': {'coordinates': [-0.3961300084192712,
     0.08640867790459059,
     0.3054716082148199],
    'species_id': 'MinDMinE',
    'state': '0'},
   '35e5a5ea-4a5b-420a-a43c-9906f9c765bf': {'coordinates': [1.691199504956,
     -0.1892058418947311,
     -0.08810059664565653],
    'species_id': 'MinDMinE',
    'state': '0'},
   '93da5fce-04c0-4cf2-ac27-f6f8b6e4a813': {'coordinates': [-0.4437098001644312,
     0.06965178275318341,
     0.1395778384725827],
    'species_id': 'MinDMinE',
    'state': '0'},
   '083d1cb4-6a2b-4811-bc5a-d152ea909c1f': {'coordinates': [1.1009701911734773,
     0.19747953364152343,
     -0.13034251953348974],
    'species_id': 'MinDMinE',
    'state': '0'},
   '4a6e4bf1-1f0e-4075-88a7-0d2e365652da': {'coordinates': [-0.7164312066970976,
     0.3565650021249067,
     0.08898093426018781],
    'species_id': 'MinDMinE

In [ ]:
data = composition.gather_results()

In [3]:
composition.state

{'smoldyn': {'_type': 'process',
  'address': 'local:smoldyn-process',
  'config': {'model_filepath': './examples/smoldyn/MinE.txt',
   'animate': False},
  'inputs': {'species_counts': ['species_counts'], 'molecules': ['molecules']},
  'outputs': {'species_counts': ['species_counts'],
   'molecules': ['molecules']},
  'instance': <biosimulator_processes.processes.smoldyn_process.SmoldynProcess at 0x1075e61a0>,
  'interval': 1.0,
  '_inputs': {'species_counts': {'MinDMinE': 'int',
    'MinD_ADP': 'int',
    'MinD_ATP': 'int',
    'MinE': 'int'},
   'molecules': 'tree[float]'},
  '_outputs': {'species_counts': {'MinDMinE': 'int',
    'MinD_ADP': 'int',
    'MinD_ATP': 'int',
    'MinE': 'int'},
   'molecules': 'tree[float]'}},
 'emitter': {'_type': 'step',
  'address': 'local:ram-emitter',
  'config': {'emit': {'species_counts': 'tree[int]',
    'molecules': 'tree[float]'}},
  'inputs': {'species_counts': ['species_counts'], 'molecules': ['molecules']},
  'instance': <process_bigraph.co